
# PyPowSyBl: Load PSS/E RAW, Run Power Flow, and Compute PTDF

1. Load PSS/E RAW
2. Run AC LF
3. Run DC LF
4. Compute PTDF


In [1]:
import pandas as pd
import pypowsybl as pp
import pypowsybl.network as pn
import pypowsybl.loadflow as lf
import pypowsybl.sensitivity as sens
from pathlib import Path

## 1. Load PSS/E RAW

In [2]:
%%time
test_system_dir = "./Test System/" # The system path goes here
RAW_PATH = Path(test_system_dir + "SmallSystem_case.raw")
network = pn.load(str(RAW_PATH), {})
buses = network.get_buses()
branches = network.get_branches()
generators = network.get_generators()
loads = network.get_loads()

CPU times: user 865 ms, sys: 43.4 ms, total: 909 ms
Wall time: 969 ms


## 2. Run AC Power Flow

In [3]:
%%time
ac_params = lf.Parameters(distributed_slack=True)
ac_results = lf.run_ac(network, parameters=ac_params)

CPU times: user 435 ms, sys: 19.4 ms, total: 454 ms
Wall time: 468 ms


## 3. Run DC Power Flow

In [4]:
%%time
dc_params = lf.Parameters(dc_use_transformer_ratio=False, distributed_slack=True,
                           balance_type=lf.BalanceType.PROPORTIONAL_TO_GENERATION_P_MAX)
dc_results = lf.run_dc(network, dc_params)

CPU times: user 87 ms, sys: 6.09 ms, total: 93.1 ms
Wall time: 92.2 ms


## 4. Compute a PTDF matrix (DC approximation)

In [8]:
%%time 

#Create sensitivity analysis
analysis = sens.create_dc_analysis()

# Include all injection points
injection_ids = []
injection_ids.extend(generators.index.tolist())
injection_ids.extend(loads.index.tolist())
branch_ids = branches.index.tolist()

# Add sensitivity matrix
analysis.add_branch_flow_factor_matrix(
    branches_ids=branch_ids,
    variables_ids=injection_ids,
    matrix_id='full_ptdf'
)

# Run analysis
result = analysis.run(network)
full_ptdf = result.get_sensitivity_matrix('full_ptdf')

CPU times: user 37.9 s, sys: 5.95 s, total: 43.8 s
Wall time: 46.1 s


## 5. Run Contingency Sensitivity Computation

In [12]:
test_system_dir = "./Test System/" # The system path goes here
RAW_PATH = Path(test_system_dir + "SmallSystem_case.raw")
network = pn.load(str(RAW_PATH), {})
buses = network.get_buses()
branches = network.get_branches()[0:1000]
generators = network.get_generators()[0:250]
loads = network.get_loads()[0:250]

In [13]:
%%time

# Create sensitivity analysis
analysis = sens.create_dc_analysis()

# Include all injection points
injection_ids = []
injection_ids.extend(generators.index.tolist())
injection_ids.extend(loads.index.tolist())

branch_ids = branches.index.tolist()

# Add sensitivity matrix
analysis.add_branch_flow_factor_matrix(
    branches_ids=branch_ids,
    variables_ids=injection_ids
)

# Set the params
parameters_lf = pp.loadflow.Parameters(
    distributed_slack=True,
    connected_component_mode=pp.loadflow.ConnectedComponentMode.MAIN,
)

# Add contingencies
for contingency in branch_ids[1:500]: # for full contingency analysis use branch_ids instead of branch_ids[1:500]
    analysis.add_single_element_contingency(contingency)

# Run analysis
result = analysis.run(network, parameters_lf)

CPU times: user 45 s, sys: 681 ms, total: 45.6 s
Wall time: 45.7 s
